In [1]:
from pathlib import Path

import numpy as np
import polars as pl
from tqdm import tqdm

debug = True
n_sampling = 9000 if debug else 650000
iter_sampling = 10 if debug else 100


output_path = Path("../misc/normalize_v3")
output_path.mkdir(exist_ok=True)

In [ ]:
df = pl.read_parquet("../input/train.parquet", n_rows=10000 if debug else None)

In [2]:
print("start y")
y_001_percentile_list = []
y_999_percentile_list = []
for i in tqdm(range(iter_sampling)):
    sample_df = df[:, 557:].sample(n_sampling, seed=i)
    y_001_percentile = sample_df.quantile(0.001).to_numpy().ravel()
    y_001_percentile_list.append(y_001_percentile)
    y_999_percentile = sample_df.quantile(0.999).to_numpy().ravel()
    y_999_percentile_list.append(y_999_percentile)

y_001_percentile_max = np.max(y_001_percentile_list, axis=0)
np.save(output_path / "y_001_percentile_max.npy", y_001_percentile)
y_999_percentile_min = np.min(y_999_percentile_list, axis=0)
np.save(output_path / "y_999_percentile_min.npy", y_999_percentile)

start y


100%|██████████| 10/10 [00:00<00:00, 37.39it/s]


In [4]:
y_diff = y_999_percentile_min - y_001_percentile_max
y_lower_bound = y_001_percentile_max - y_diff
np.save(output_path / "y_lower_bound.npy", y_lower_bound)

y_upper_bound = y_999_percentile_min + y_diff
np.save(output_path / "y_upper_bound.npy", y_upper_bound)

In [6]:
print(
    f"{y_001_percentile.shape=}, {y_999_percentile.shape=}, {y_lower_bound.shape=}, {y_upper_bound.shape=}"
)

y = df[:, 557:].to_numpy()
y[y < y_lower_bound] = np.nan
y[y > y_upper_bound] = np.nan
y_nanmean = np.nanmean(y, axis=0)
np.save(output_path / "y_nanmean.npy", y_nanmean)
y_nanmin = np.nanmin(y, axis=0)
np.save(output_path / "y_nanmin.npy", y_nanmin)
y_nanmax = np.nanmax(y, axis=0)
np.save(output_path / "y_nanmax.npy", y_nanmax)
y_nanstd = np.nanstd(y, axis=0)
np.save(output_path / "y_nanstd.npy", y_nanstd)

y_001_percentile.shape=(368,), y_999_percentile.shape=(368,), y_lower_bound.shape=(368,), y_upper_bound.shape=(368,)


In [18]:
eps = 1e-60
filter_bool = np.all(df[:, 557:].to_numpy() >= y_lower_bound - eps, axis=1) & np.all(
    df[:, 557:].to_numpy() <= y_upper_bound + eps, axis=1
)
print(np.sum(df[:, 557:].to_numpy() > y_lower_bound, axis=1))
print(np.sum(df[:, 557:].to_numpy() >= y_lower_bound - eps, axis=1))

[306 306 306 ... 306 306 306]
[368 368 368 ... 368 368 368]


In [12]:
np.all(df[:, 557:].to_numpy() >= y_lower_bound - eps, axis=1)

array([ True,  True,  True, ...,  True,  True,  True])

In [13]:
np.all(df[:, 557:].to_numpy() <= y_upper_bound + eps, axis=1)

array([ True,  True,  True, ...,  True,  True,  True])

In [15]:
filter_bool

array([ True,  True,  True, ...,  True,  True,  True])

In [16]:

print(f"{df.shape=}")
df = df.filter(filter_bool)

df.shape=(10000, 925)


In [17]:
print(f"{df.shape=}")


df.shape=(9884, 925)
